## Часть 2



- Я хочу проанализировать все возможные характеристики для KNN-графа и выбрать, какая из них работает лучше. Это минимальная и макимальная степени, число точек сочленения, число треугольников и число компонент связности.

In [33]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'src')))

import importlib
import functions
importlib.reload(functions)

<module 'functions' from '/Users/berdov/dm_pr/dm/src/functions.py'>

In [34]:
from functions import max_degree
from functions import min_degree
from functions import count_articulation_points
from functions import count_triangles
from functions import count_components

- Теперь я подключу функции генерации распределений и многократного проведения эксперимента


In [35]:
from functions import (
    build_knn_graph,
    sample_normal,
    sample_t,
    monte_carlo_characteristic
)

- Теперь все готово к анализу, какая же характеристика лучше

In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt


characteristics = {
    'Макс. степень': max_degree,
    'Мин. степень': min_degree,
    'Компоненты': count_components,
    'Сочленения': count_articulation_points,
    'Треугольники': count_triangles
}

summary = []

n = 100
k = [1, 3, 5, 10, 20, 40, 60]
n_sim = 200
alpha = 0.05


summary = []

for k in k:
    for name, func in characteristics.items():
        print(f"Анализ: {name}")
        lambda_H0 = 0.5**0.5
        lambda_H1 = 1
        X0_raw = monte_carlo_characteristic(sample_normal, lambda X: build_knn_graph(X, k), func, 1, n=n, n_sim=n_sim)
        X1_raw = monte_carlo_characteristic(sample_t, lambda X: build_knn_graph(X, k), func, 3, n=n, n_sim=n_sim)
        if np.median(X0_raw) < np.median(X1_raw):
            T_H0, T_H1 = X0_raw, X1_raw
        else:
            T_H0, T_H1 = X1_raw, X0_raw

        threshold = np.percentile(T_H0, 100 * (1 - alpha))
        power = np.mean(T_H1 > threshold)
        auc = roc_auc_score([0]*len(T_H0) + [1]*len(T_H1), np.concatenate([T_H0, T_H1]))
        summary.append({
            'k': k,
            'Характеристика': name,
            'AUC ROC': round(auc, 3),
            'Power (H1)': round(power, 3),
            'Порог (95% H0)': round(threshold, 2)
        })

df_summary = pd.DataFrame(summary)
print(df_summary)


Анализ: Макс. степень
Анализ: Мин. степень
Анализ: Компоненты
Анализ: Сочленения
Анализ: Треугольники
Анализ: Макс. степень
Анализ: Мин. степень
Анализ: Компоненты
Анализ: Сочленения
Анализ: Треугольники
Анализ: Макс. степень
Анализ: Мин. степень
Анализ: Компоненты
Анализ: Сочленения
Анализ: Треугольники
Анализ: Макс. степень
Анализ: Мин. степень
Анализ: Компоненты
Анализ: Сочленения
Анализ: Треугольники
Анализ: Макс. степень
Анализ: Мин. степень
Анализ: Компоненты
Анализ: Сочленения
Анализ: Треугольники
Анализ: Макс. степень
Анализ: Мин. степень
Анализ: Компоненты
Анализ: Сочленения
Анализ: Треугольники
Анализ: Макс. степень
Анализ: Мин. степень
Анализ: Компоненты
Анализ: Сочленения
Анализ: Треугольники
     k Характеристика  AUC ROC  Power (H1)  Порог (95% H0)
0    1  Макс. степень    0.500       0.000            2.00
1    1   Мин. степень    0.500       0.000            1.00
2    1     Компоненты    0.526       0.055           36.00
3    1     Сочленения    0.482       0.015        

# Вывод для KNN-графа

- отвратительно отработали абсолютно все арактеристики, кроме числа треугольников, причем этот вывод почти не меняется в зависимости от разного K. При больших K критерий число треугольников становится все более точным, в то время как остальные характеристики становятся все менее отличимы. буду все переделывать. Теперь буду строить DIST-граф и считать у него хроматическое число, кликовое число, размер максимального независимого множества, доминирующее число, размер минимального кликового покрытия

In [37]:
from functions import chromatic_number
from functions import clique_number
from functions import max_independent_set_size
from functions import domination_number
from functions import clique_cover_number

- повторим анализ аналогичным образом

In [38]:
from functions import (
    build_dist_graph,
    sample_normal,
    sample_t,
    monte_carlo_characteristic
)


from networkx.algorithms import approximation as approx

In [39]:
characteristics = {
    'Хроматическое число': chromatic_number,
    'Кликовое число': clique_number,
    'Макс. независимое множество': max_independent_set_size,
    'Доминирование': domination_number,
    'Кликовое покрытие': clique_cover_number
}

summary = []

n = 100
d = [0.1, 0.3, 0.5, 1, 2]
n_sim = 200
alpha = 0.05


summary = []

for d in d:
    for name, func in characteristics.items():
        print(f"Анализ: {name}")
        lambda_H0 = 0.5**0.5
        lambda_H1 = 1
        X0_raw = monte_carlo_characteristic(sample_normal, lambda X: build_dist_graph(X, d), func, 1, n=n, n_sim=n_sim)
        X1_raw = monte_carlo_characteristic(sample_t, lambda X: build_dist_graph(X, d), func, 3, n=n, n_sim=n_sim)
        if np.median(X0_raw) < np.median(X1_raw):
            T_H0, T_H1 = X0_raw, X1_raw
        else:
            T_H0, T_H1 = X1_raw, X0_raw

        threshold = np.percentile(T_H0, 100 * (1 - alpha))
        power = np.mean(T_H1 > threshold)
        auc = roc_auc_score([0]*len(T_H0) + [1]*len(T_H1), np.concatenate([T_H0, T_H1]))
        summary.append({
            'd': d,
            'Характеристика': name,
            'AUC ROC': round(auc, 3),
            'Power (H1)': round(power, 3),
            'Порог (95% H0)': round(threshold, 2)
        })

df_summary = pd.DataFrame(summary)
print(df_summary)



Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
      d               Характеристика  AUC ROC  Power (H1)  Порог (95% H0)
0   0.1          Хроматическое число    0.638       0.125           10.05
1   0.1               Кликовое число    0.642       0.040           11.00
2   0.1  Макс. независимое множество    0.975       0.835           30.00
3   0.1                Домини

# Вывод для DIST - графа

- до чего же хорошие распределения мне выдали проверяющие. У всех моих экспериментов есть только 1 критерий, который хорошо отвечает на поставленную задачу: размер максимального незавимисого множества в DIST-графе. У него мощность = 1, остальные критерии справились очень плохо. Очевидно, для обучения модели надо выбрать DIST-граф. Обратим внимание, что с ростом d качество параметра доминирование растет существенно быстрее остальных. Например, при d = 2 параметр показал такую же эффективность, как и размер максимального независимого множества. 

# Оценим важность характеристик с ростом n, зафиксируем d = 2.

In [42]:

summary = []

n = [10, 25, 100, 200]
n_sim = 200
alpha = 0.05
d = 2

summary = []

for n in n:
    for name, func in characteristics.items():
        print(f"Анализ: {name}")
        lambda_H0 = 0.5**0.5
        lambda_H1 = 1
        X0_raw = monte_carlo_characteristic(sample_normal, lambda X: build_dist_graph(X, d), func, 1, n=n, n_sim=n_sim)
        X1_raw = monte_carlo_characteristic(sample_t, lambda X: build_dist_graph(X, d), func, 3, n=n, n_sim=n_sim)
        if np.median(X0_raw) < np.median(X1_raw):
            T_H0, T_H1 = X0_raw, X1_raw
        else:
            T_H0, T_H1 = X1_raw, X0_raw

        threshold = np.percentile(T_H0, 100 * (1 - alpha))
        power = np.mean(T_H1 > threshold)
        auc = roc_auc_score([0]*len(T_H0) + [1]*len(T_H1), np.concatenate([T_H0, T_H1]))
        summary.append({
            'n': n,
            'Характеристика': name,
            'AUC ROC': round(auc, 3),
            'Power (H1)': round(power, 3),
            'Порог (95% H0)': round(threshold, 2)
        })

df_summary = pd.DataFrame(summary)
print(df_summary)



Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
      n               Характеристика  AUC ROC  Power (H1)  Порог (95% H0)
0    10          Хроматическое число    0.676       0.065            9.00
1    10               Кликовое число    0.664       0.055            9.00
2    10  Макс. независимое множество    0.325       0.000            3.00
3    10                Доминирование    0.754       0.370            6.00
4    10            Кликовое покрытие    0.672       0.090            9.00
5    25         

# Вывод при росте n


- при фиксации правильного d = 2 получили не 1, а 2 хороших критерия для классификации: доминирование и размер макс независимого множества. Остальные тоже показали эффективность, причем ненулевую.

# Обучение моделей


- первым делом подключим функцию для создания датасета и собственно говоря сгенерируем его


In [43]:
from functions import generate_dataset

- Я принял решение сравнить 3 модели: SVM, случайный лес и логическую регрессию. Проанализируем показатели при n = 25, 100, 500.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import numpy as np


def evaluate_type1_and_power(clf, df_train, df_H0, df_H1):
    X_train = df_train.drop("label", axis=1)
    y_train = df_train["label"]
    clf.fit(X_train, y_train)

    y_pred_H0 = clf.predict(df_H0.drop("label", axis=1))
    y_pred_H1 = clf.predict(df_H1.drop("label", axis=1))

    type1_error = (y_pred_H0 == 1).mean()
    power = (y_pred_H1 == 1).mean()
    return type1_error, power


models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM (RBF)": SVC(probability=True)
}

for n_val in [25, 100, 500]:
    df_train = generate_dataset(
        n=n_val,
        d=0.3,
        dist_H0=sample_normal,
        args_H0=(1,),
        dist_H1=sample_t,
        args_H1=(3,),
        feature_funcs=[max_independent_set_size, domination_number, clique_number, chromatic_number, clique_cover_number],
        graph_func=lambda X: build_dist_graph(X, 0.3),
        n_sim=300
    )
    X = df_train.drop("label", axis=1)
    y = df_train["label"]

    df_H0 = generate_dataset(n=n_val, d=0.3, dist_H0=sample_normal, args_H0=(1,),
                             dist_H1=sample_t, args_H1=(3,),
                             feature_funcs=[max_independent_set_size, domination_number, clique_number, chromatic_number, clique_cover_number],
                             graph_func=lambda X: build_dist_graph(X, 0.3),
                             n_sim=100).query("label == 0")

    df_H1 = generate_dataset(n=n_val, d=0.3, dist_H0=sample_normal, args_H0=(1,),
                             dist_H1=sample_t, args_H1=(3,),
                             feature_funcs=[max_independent_set_size, domination_number, clique_number, chromatic_number, clique_cover_number],
                             graph_func=lambda X: build_dist_graph(X, 0.3),
                             n_sim=100).query("label == 1")

    for name, model in models.items():
        auc = cross_val_score(model, X, y, cv=5, scoring="roc_auc")
        acc = cross_val_score(model, X, y, cv=5, scoring="accuracy")
        clf = model.__class__(**model.get_params())
        type1_error, power = evaluate_type1_and_power(clf, df_train, df_H0, df_H1)

        print()
        print(f"n={n_val}, {name}:")
        print(f"  AUC     = {auc.mean():.3f} ± {auc.std():.3f}")
        print(f"  Accuracy= {acc.mean():.3f} ± {acc.std():.3f}")
        print(f"  Type I Error = {type1_error:.3f}")
        print(f"  Power        = {power:.3f}")


n=25, Random Forest:
  AUC     = 0.760 ± 0.024
  Accuracy= 0.693 ± 0.030
  Type I Error = 0.250
  Power        = 0.620

n=25, Logistic Regression:
  AUC     = 0.842 ± 0.033
  Accuracy= 0.767 ± 0.045
  Type I Error = 0.270
  Power        = 0.650

n=25, SVM (RBF):
  AUC     = 0.840 ± 0.034
  Accuracy= 0.750 ± 0.038
  Type I Error = 0.280
  Power        = 0.640

n=100, Random Forest:
  AUC     = 0.973 ± 0.021
  Accuracy= 0.937 ± 0.027
  Type I Error = 0.020
  Power        = 0.910

n=100, Logistic Regression:
  AUC     = 0.990 ± 0.010
  Accuracy= 0.955 ± 0.015
  Type I Error = 0.030
  Power        = 0.910

n=100, SVM (RBF):
  AUC     = 0.983 ± 0.017
  Accuracy= 0.938 ± 0.039
  Type I Error = 0.010
  Power        = 0.890

n=500, Random Forest:
  AUC     = 1.000 ± 0.000
  Accuracy= 0.998 ± 0.003
  Type I Error = 0.000
  Power        = 1.000

n=500, Logistic Regression:
  AUC     = 1.000 ± 0.000
  Accuracy= 0.998 ± 0.003
  Type I Error = 0.000
  Power        = 1.000

n=500, SVM (RBF):
  AUC 

# Выводы при n = 25, 100, 500

- c ростом n качество повышается: растет AUC, accuracy, power. Две модели из трех справились неплохо. О них чуть ниже.

# Лучшая модель - классическая логическая регрессия.

- логрег и случайный лес справились почти одинаково, но на n=100 был небольшой перевес логрега, поэтому я выбрал именно ее. При n = 25 accuracy составила 0.75, при n = 100: 0.95 и при n =500: 1.0

- вероятность ошибки на разных выборках составила 0.27, 0.03 и 0. Дисперсии также на достйном уровне(показаны в выходном окошке).

# ВСЕ!